In [62]:

import os
import re
import pandas as pd
import configparser
import jaconv
from tqdm import tqdm
import math 

In [33]:
config = configparser.ConfigParser()
config.read("../config.ini")

nutrition_data_path = config.get("Data","nutrition_data_path")
OUTPUT_ROOT = config.get("Output","output_path")

In [34]:
def load_nutrition_data_from_folder(base_path, subfolder, column_mapping, header_row=4, sheet_name="表全体"):
    # フォルダパスを構築
    folder_path = os.path.join(base_path, subfolder)
    
    # このフォルダ内の全てのxlsxファイルを取得
    files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
    
    # 全データを格納する空のデータフレームを初期化
    all_nutrition_data = pd.DataFrame()
    
    for file in files:
        # ファイルからデータを読み込む
        file_path = os.path.join(folder_path, file)
        data = pd.read_excel(file_path, header=[header_row], sheet_name=sheet_name)
        
        # 列の名前を変更
        data = data.rename(columns=column_mapping)
        
        # 'food_code'がNAの行を削除
        data.dropna(subset=['food_code'], inplace=True)
        
        # 全データフレームにこのデータを追加
        all_nutrition_data = pd.concat([all_nutrition_data, data], ignore_index=True)
    
    return all_nutrition_data

### 成分表結合

In [35]:
column_mapping = {
    "Unnamed: 0":"food_group",
    "Unnamed: 1":"food_code",
    "Unnamed: 2":"reference_number",
    "成分識別子":"food_name",
    "Unnamed: 61":"note_all"
}

In [36]:
nutrition_data = load_nutrition_data_from_folder(nutrition_data_path,"mtx_01",column_mapping,header_row=11)


## アミノ酸成分表

In [37]:
amino_column_mapping = {
    "Unnamed: 0":"food_group",
    "Unnamed: 1":"food_code",
    "Unnamed: 2":"reference_number",
    "成分識別子":"food_name",
    "Unnamed: 31":"note_amino"
}

In [38]:
amino_nutrition_data = load_nutrition_data_from_folder(nutrition_data_path,"mtx_02",amino_column_mapping,header_row=4)

## 脂肪酸成分表

In [39]:
fat_column_mapping = {
    "Unnamed: 0":"food_group",
    "Unnamed: 1":"food_code",
    "Unnamed: 2":"reference_number",
    "成分識別子":"food_name",
    "Unnamed: 31":"note_fat"
}

In [40]:
fat_nutrition_data = load_nutrition_data_from_folder(nutrition_data_path,"mtx_03",fat_column_mapping,header_row=4)

## 炭水化物成分表

In [41]:
carb_column_mapping = {
    "Unnamed: 0":"food_group",
    "Unnamed: 1":"food_code",
    "Unnamed: 2":"reference_number",
    "成分識別子":"food_name",
    "Unnamed: 17":"note_carb"
}

In [42]:
carb_nutrition_data = load_nutrition_data_from_folder(nutrition_data_path,"mtx_04",carb_column_mapping,header_row=4)

## まとめ

In [43]:
print("食品数（成分表）:",len(nutrition_data))
print("食品数（アミノ酸成分表）:",len(amino_nutrition_data))
print("食品数（脂肪酸成分表）:",len(fat_nutrition_data))
print("食品数（炭水化物成分表）:",len(carb_nutrition_data))

食品数（成分表）: 2538
食品数（アミノ酸成分表）: 4525
食品数（脂肪酸成分表）: 4523
食品数（炭水化物成分表）: 2995


In [44]:

all_frames = [nutrition_data, amino_nutrition_data, fat_nutrition_data, carb_nutrition_data]

all_nutrition_data = pd.concat(all_frames, axis=0, ignore_index=True)

def clean_and_classify(row):
    original = row  # オリジナルの文字列を保持
    if not isinstance(row, str) or row == '':
        return {
            'food_name_cleaned': '',
            'sub_category': '',
            'type_category': '',
            'mid_category': '',
            'small_category': ''
        }
    # 同じ処理を続けます...
    sub_category = re.search(r'＜(.*?)＞', row)
    row = re.sub(r'＜(.*?)＞', '', row)

    type_category = re.search(r'（(.*?)）', row)
    row = re.sub(r'（(.*?)）', '', row)

    mid_category = re.search(r'［(.*?)］', row)
    row = re.sub(r'［(.*?)］', '', row)

    small_category = row.strip()

    return {
        'food_name_cleaned': original,
        'sub_category': sub_category.group(1) if sub_category else '',
        'type_category': type_category.group(1) if type_category else '',
        'mid_category': mid_category.group(1) if mid_category else '',
        'small_category': small_category
    }

# 新しい列を追加
all_nutrition_data = all_nutrition_data.join(all_nutrition_data['food_name'].apply(lambda row: pd.Series(clean_and_classify(row))))

def katakana_to_hiragana(name):
    if isinstance(name, str):  # nameが文字列かどうかを確認
        return jaconv.kata2hira(name)
    else:
        return name
    
all_nutrition_data.dropna(subset=['food_name_cleaned'], inplace=True)
all_nutrition_data["food_name_cleaned"] = all_nutrition_data["food_name_cleaned"].apply(katakana_to_hiragana)
all_nutrition_data.dropna(subset=['food_code'], inplace=True)

# floatからintに変換（NaNがなくなったので問題なく変換できる）
all_nutrition_data['food_code'] = all_nutrition_data['food_code'].astype(int)

# intからstrに変換
all_nutrition_data['food_code'] = all_nutrition_data['food_code'].astype(str)

all_nutrition_data

,food_group,food_code,reference_number,food_name,REFUSE,ENERC,ENERC_KCAL,WATER,PROTCAA,PROT-,...,QUINAC,OROTAC,PYROGAC,PROPAC,Unnamed: 28,food_name_cleaned,sub_category,type_category,mid_category,small_category
0,1.0,1001,0001,アマランサス 玄穀,0.0,1452.0,343.0,13.5,(11.3),12.7,...,NaN,NaN,NaN,NaN,NaN,あまらんさす 玄穀,,,,アマランサス 玄穀
1,1.0,1002,0002,あわ 精白粒,0.0,1466.0,346.0,13.3,10.2,11.2,...,NaN,NaN,NaN,NaN,NaN,あわ 精白粒,,,,あわ 精白粒
2,1.0,1003,0003,あわ あわもち,0.0,890.0,210.0,48.0,(4.5),5.1,...,NaN,NaN,NaN,NaN,NaN,あわ あわもち,,,,あわ あわもち
3,1.0,1004,0004,えんばく オートミール,0.0,1479.0,350.0,10.0,12.2,13.7,...,NaN,NaN,NaN,NaN,NaN,えんばく おーとみーる,,,,えんばく オートミール
4,1.0,1005,0005,おおむぎ 七分つき押麦,0.0,1454.0,343.0,14.0,(9.7),10.9,...,NaN,NaN,NaN,NaN,NaN,おおむぎ 七分つき押麦,,,,おおむぎ 七分つき押麦
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14576,18.0,18053,2446-1,和風料理 その他 お好み焼き,NaN,NaN,NaN,71.8,NaN,NaN,...,-,-,-,-,NaN,和風料理 その他 お好み焼き,,,,和風料理 その他 お好み焼き
14577,18.0,18054,2446-2,和風料理 その他 とりから揚げ,NaN,NaN,NaN,55.8,NaN,NaN,...,-,-,-,-,NaN,和風料理 その他 とりから揚げ,,,,和風料理 その他 とりから揚げ
14578,18.0,18055,2464-1,洋風料理 フライ類 かきフライ,NaN,NaN,NaN,46.3,NaN,NaN,...,-,-,-,-,NaN,洋風料理 ふらい類 かきふらい,,,,洋風料理 フライ類 かきフライ
14579,18.0,18056,2477-1,中国料理 点心類 春巻き,NaN,NaN,NaN,42.8,NaN,NaN,...,-,-,-,-,NaN,中国料理 点心類 春巻き,,,,中国料理 点心類 春巻き


In [56]:
drop_col_ls = [
    'food_group',
    'food_code',
    'reference_number',
    'food_name_cleaned',
    'Unnamed: 14',
    'Unnamed: 17',
    'Unnamed: 32',
    'note_all',
    'note_amino',
    'Unnamed: 29',
    'Unnamed: 27',
    'Unnamed: 62',
    'Unnamed: 58',
    'Unnamed: 59',
    'note_carb',
    'Unnamed: 3',
    'Unnamed: 4',
    'プロスキー変法',
    'Unnamed: 6',
    'Unnamed: 7',
    'AOAC.2011.25法',
    'Unnamed: 9',
    'Unnamed: 10',
    'Unnamed: 11',
    'Unnamed: 12',
    'Unnamed: 13',
    'Unnamed: 28',
    'sub_category',
    'type_category',
    'mid_category',
    'small_category',
    ]
trg_col_ls = list(set(all_nutrition_data.columns) - set(drop_col_ls))

In [66]:
nutrition_edge_mtx = []
for index,row in tqdm(all_nutrition_data.iterrows(),total = len(all_nutrition_data)):
    food_name = row["food_name_cleaned"]
    food_code = row["food_code"]
    for nut_col in trg_col_ls:
        trg_value = row[nut_col]
        if isinstance(trg_value, (int, float)) and not math.isnan(trg_value):
            add_nutrition_edge_ls = [food_code,food_name,nut_col,trg_value,"ingredient-nutrition","seibunhyo"]
            nutrition_edge_mtx.append(add_nutrition_edge_ls)
nutrition_edge_df = pd.DataFrame(nutrition_edge_mtx,columns=["food_code","food_name","nutrition_name","value","edge_type","data_source"])

100%|██████████| 14581/14581 [00:12<00:00, 1214.64it/s]


In [67]:
save_path = os.path.join(OUTPUT_ROOT,"output_csv","seibunhyo_edges.csv") 
nutrition_edge_df.to_csv(save_path,index=False)

In [68]:
nutrition_edge_df

,food_code,food_name,nutrition_name,value,edge_type,data_source
0,1001,あまらんさす 玄穀,PANTAC,1.69,ingredient-nutrition,seibunhyo
1,1001,あまらんさす 玄穀,ENERC_KCAL,343.00,ingredient-nutrition,seibunhyo
2,1001,あまらんさす 玄穀,CR,7.00,ingredient-nutrition,seibunhyo
3,1001,あまらんさす 玄穀,CU,0.92,ingredient-nutrition,seibunhyo
4,1001,あまらんさす 玄穀,VITB6A,0.58,ingredient-nutrition,seibunhyo
...,...,...,...,...,...,...
266018,18056,中国料理 点心類 春巻き,CITAC,0.10,ingredient-nutrition,seibunhyo
266019,18056,中国料理 点心類 春巻き,WATER,42.80,ingredient-nutrition,seibunhyo
266020,18056,中国料理 点心類 春巻き,OA,0.20,ingredient-nutrition,seibunhyo
266021,18057,中国料理 菜類 ちゃーはん,WATER,55.10,ingredient-nutrition,seibunhyo
